In [1]:
from utils import *
print('Project root directory:', PROJ_DIR)

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import os

dir = os.path.join(PROJ_DIR, BIODATA_DIR, 'drosophila_master')
tf_master_data_filename = 'result_upstream12k_downstream6k_hkbadd.xls'
repressors_filename = 'repressors.xls'

SITE_START = 'loc. start'
SITE_END = 'loc. end'

# load data
data = pd.read_excel(os.path.join(dir, tf_master_data_filename), sheet_name=None)
repressors = pd.read_excel(os.path.join(dir, repressors_filename), index_col=0)
# lowercase protein and gene names
data = {key.lower(): val for key, val in data.items()}
repressors.index = repressors.index.str.lower()
repressors.columns = repressors.columns.str.lower()
# some useful arrays
tf_list = repressors.columns[1:].to_list()
genes_list = repressors.index.to_list()

Project root directory: /home/fedor/Documents/GRiPE


In [12]:
def is_repressor(gene_name, tf_name):
    return bool(repressors.loc[gene_name, tf_name])

def func(data, gene, site_i, radius, pattern, start, end, id):
    if site_i < 0:
        return 0
    # select sites in radius
    site_pos = data.loc[site_i][SITE_START]
    site_end = data.loc[site_i][SITE_END]
    cond1 = data[SITE_END] > site_pos - radius
    cond2 = data[SITE_START] < site_end + radius
    sites_in_rad = data.loc[cond1 & cond2]
    # add these sites to the lists and save the last repressor
    last_repr_id = -1
    n = 0
    for i, site in sites_in_rad.iterrows():
        if not (i in id): # if the current site is not in the list
            if len(end) > 0 and site.loc[SITE_START] < end[-1]:
                pattern.append('/') # indicates overlap of sites
            if is_repressor(gene, site.loc['tf']):
                pattern.append('R')
                last_repr_id = i
            else:
                pattern.append('A')
            if len(start) > 0 and site.loc[SITE_START] > data.loc[site_i][SITE_START]:
                n += 1
            id.append(i)
            start.append(site.loc[SITE_START])
            end.append(site.loc[SITE_END])
    # call this func with the last repressor
    #print(pattern, start, end, id)
    n1 = func(data, gene, last_repr_id, radius, pattern, start, end, id)
    # return the number of new sites to the right of the current one
    return n + n1

In [20]:
repr_radius = 2
print('site_patterns_rad_{}.xlsx'.format(repr_radius))

site_patterns_rad_2.xlsx


In [21]:
for repr_radius in [10, 50, 100]:
    writer = pd.ExcelWriter(os.path.join(dir, 'site_patterns_rad_{}.xlsx'.format(repr_radius)), engine="xlsxwriter")
    for gene in genes_list:
        site_groups = pd.DataFrame(columns=['pattern', 'strand', 'start', 'end', 'id'])
        data_sorted = data[gene].sort_values(SITE_START)
        for strand in ['+', '-']:
            data_sorted_strand = data_sorted.loc[data_sorted['loc. strand'] == strand]
            iter = data_sorted_strand.iterrows()
            for site_i, site in iter:
                if is_repressor(gene, site['tf']):
                    pattern = []
                    start = []
                    end = []
                    id = []
                    n = func(data_sorted_strand, gene, site_i, repr_radius, pattern, start, end, id)
                    for _ in range(n):
                        next(iter)
                    #print("".join(pattern), len(start), len(id), n)
                    site_groups = site_groups.append({'pattern': "".join(pattern), 'start': np.array(start), 'strand': strand,
                                                    'end': np.array(end), 'id': np.array(id)}, ignore_index=True)
        site_groups.to_excel(writer, sheet_name=gene, index=False)
    writer.save()